# Régression logistique - Exercices tirés du MOOC d'Andrew Ng

## Chargement des données

Le jeu de données représente les chances de différents étudiants d'être admis à un programme universitaire en fonction de leurs résultats à deux examens

### Charger les données du fichier ex2data1.csv

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook


In [2]:
data = pd.read_csv("ex2data1.csv")

### Visualiser les données

Cette semaine nous vous fournissons la visualisation des données, mais normalement vous auriez à produire un graphe semblable avec matplotlib.

<img src="figure-1.png">

### Découpez vos données en une matrice X et un vecteur y et transformerz-les en array numpy

In [3]:
X = np.insert(data[['exam1', 'exam2']].values, 0, 1, axis=1)
y = data['admission'].values.reshape((-1, 1))
print(X.shape, y.shape)

(100, 3) (100, 1)


Vérifiez bien les dimensions de vos structures de données (X.shape)  
Rappelez-vous qu'il est judicieux de fixer les dimensions des vecteurs, par ex. (3,) avec la fonction reshape(3,1).
La matrice X doit-elle être de dimensions m x n ou bien m x (n+1) ? Quelle est la valeur de n?

### Initialisez theta en un vecteur de zéros

Combien de zéros vous faudra-t-il....?

In [4]:
theta = np.zeros((X.shape[1], 1))
print(theta.shape)

(3, 1)


## Formulation de l'hypothèse

Revoyez l'équation de l'hypothèse de la régression logistique. Le produit de theta et de X est enveloppé dans une fonction g(z) qui correspond à la fonction sigmoïde. Nous allons commencer par coder cette fonction.

### Écrivez une fonction _sigmoid_ qui applique la sigmoïde à son argument et retourne le résultat. Si elle reçoit une matrice ou un vecteur en input, elle doit s'appliquer sur chaque élément individuellement et retourner une structure de mêmes dimensions

In [5]:
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

np.set_printoptions(precision=20)
print(sigmoid(25))

0.999999999986112


Vérifiez votre fonction. Quelle valeur renvoie une sigmoïde si z=0? Si z est grand? Si z est petit?  
Il est possible que vous ayez un bug lorsque la fonction exponentielle reçoit des valeurs trop grandes. Dans ce cas vous pourrez éventuellement remplacer votre fonction sigmoïde par celle de scipy pour éviter des problèmes dans le reste de votre implémentation.

### Écrivez une fonction _predict_, qui correspond à l'hypohèse hθ(x), qui prend en paramètres X et theta, applique l'hypothèse du modèle avec la fonction sigmoide, et se débrouille pour que le résultat final soit un vecteur de 1 et de 0 correspondant aux catégories

In [6]:
def predict(X, theta):
    z = sigmoid(np.dot(X, theta))
    # on veut une asymptote donc on ne veut pas de valeur en 0 ou 1,
    # donc on retire/ajoute 0.00...001 pour ne pas avoir de problème avec
    # log(0) => erreur
    # on peut aussi éviter ça avec le parameters scaling
    z[(z == 0)] += 10e-10 
    z[(z == 1)] -= 10e-10
    return (z)

print(predict(X, theta).T)

[[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
  0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
  0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
  0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
  0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
  0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]]


## Entraînement du modèle

### Définissez la fonction de coût de votre modèle

In [7]:
def cost(X, y, theta):
    h = predict(X, theta)
    return (np.dot(-y.T, np.log(h)) - (np.dot((1 - y).T, np.log(1 - h))))[0][0] / X.shape[0]

### Calculez le coût de votre modèle non entraîné. Vous devriez obtenir une valeur d'environ 0.693

In [8]:
print(cost(X, y, theta))

0.6931471805599453


### Écrivez une fonction _fit_ qui prend en arguments le vecteur X et le vecteur y des données d'entraînement et renvoie le vecteur de paramètres _theta_ qui a été appris, ainsi que l'évolution du coût

Noter que l'exercice original ne fait pas faire la descente du gradient pour entraîner le modèle, mais plutôt une fonction d'optimisation avancée (_fminunc_ en Matlab). Nous tenterons de faire quand même la descente du gradient. Les plus téméraires peuvent aussi trouver une fonction d'optimisation équivalente en Python et comparer les résultats.

In [9]:
# nouvelle fonction de coût pour éviter de faire predict() plusieurs fois
def cost_predict(h, y):
    return (np.dot(-y.T, np.log(h)) - (np.dot((1 - y).T, np.log(1 - h))))[0][0] / X.shape[0]

def get_reg(lmbdpersize, theta, methode):
    if methode == "L1":
        reg = lmbdpersize * np.abs(theta)
    else:
        reg = lmbdpersize * np.square(theta)
    reg[0] = theta[0]
    return reg

def fit(X, y, theta, alpha, lmbd, num_iters, reg=""):
    # alpha / m, on le fait au début pour le calculer qu'une seule fois
    coef = alpha / X.shape[0]
    lmbdpersize = lmbd / X.shape[0]
    # copie de theta pour ne pas ensuite modifier l'ancien pointeur
    theta = theta.copy()
    # 1ere prediction et calcul du cout
    h = predict(X, theta)
    J_history = [cost_predict(h, y)]
    # boucle
    for i in tqdm_notebook(range(num_iters)):
        # mise à jour de theta
        theta -= coef * (np.dot(X.T, h - y) + get_reg(lmbdpersize, theta, reg))                      
        # nouvelle prediction
        h = predict(X, theta)
        J_history.append(cost_predict(h, y))
        # si on veut sortir dès une convergeance de la fonction de cout
        # mais peu utile pour notre cas
        # if (J_history[i] - J_history[i + 1] < 10e-8).all():
        #    break    
    return theta, np.array(J_history)

### Lancez l'apprentissage en appelant la fonction _fit_ et en prenant bien soin de récupérer le résultat de *theta* à la fin!!

Voyez entre vous quelles valeurs semblent correctes pour alpha et num_iters

In [10]:
theta2, J_history = fit(X, y, theta, 0.0001, 0.01, 10000000, "L2")
print(J_history)
print(theta2)


[0.6931471805599453  0.6690967609080579  0.6546428926944947  ...
 0.4203381168164085  0.4203381168163933  0.42033811681637817]
[[-3.9003499101395342  ]
 [ 0.03845072202458161 ]
 [ 0.031022411414359352]]


### Appelez la fonction _cost_ avec le nouveau theta après entraînement

Vous devriez obtenir une valeur autour de 0.203

In [11]:
cost(X, y, theta2)

0.42033811681637817

### On visualise maintenant l'évolution du coût en fonction du nombre d'itérations

In [12]:
import matplotlib.pyplot as plt
fig = plt.figure()
ax = plt.axes()
ax.plot(J_history)

## Évaluation de votre modèle

Nous allons évaluer la performance du modèle de deux façons:

### Évaluez la probabilité qu'un étudiant ayant obtenu 45 au premier examen, et 85 au deuxième, soit admis

Vous devriez avoir une probabilité d'admission de 0.776

In [13]:
print(predict([[1, 45, 85]], theta2))

[[0.6146349745666861]]


### Évaluer l'exactitude (accuracy) des prédictions faites sur les données d'entraînement

Utilisez votre fonction _predict_ sur les données d'entraînement (X) et récupérez les prédictions dans un vecteur p

In [14]:
p = predict(X, theta2)

Calculez le pourcentage des éléments de p qui correspondent à ceux de y. Ça vous donne le score d'exactitude

In [15]:
# on remplit p de 0 et de 1 selon si le seuil des 0.5 est franchi
p = np.where(p > 0.5, 1, 0)
# somme des éléments égaux entre p et y, divisé par le nombre d'éléments
print(np.sum(p == y) / p.shape[0])


0.87


Vous devriez avoir un score d'environ 89.0 %

### Quelle est la précision, le recall et le F1-score de votre modele ? (écrivez trois fonctions pour obtenir chacunes de ces métriques)

In [16]:
# pour rappel :
# true/false positive/negative

# positive/nagative, c'est la prediction, si elle est à 1 ou 0 (p)
# true/ false, c'est si cette prediction est juste ou pas (p == y)

# donc remplacez l'expression par l'equation correspondante si vous vous y perdez

#                   ( (p == y) p )
# true  positive => ( (1 == 1) 1 ) => les predictions Oui et justes
# false positive => ( (1 == 0) 1 ) => les predictions Oui dans l'erreur
# true  negative => ( (0 == 0) 0 ) => les predictions Non et justes
# false negative => ( (0 == 1) 0 ) => les predictions Non dans l'erreur

def precision(p, y):
    # somme des true positives
    # on récupère dans y les valeurs correspondants aux prédictions positives de p
    # et on fait la somme des valeurs à 1 pour avoir le nombre de true positives.
    tp = np.sum(y[(p == 1)])
    # predictions positives (true positive + false positive)
    # somme des valeurs à 1 de p
    pp = np.sum(p)
    return tp / pp
    
def recall(p, y):
    # true positives encore
    tp = np.sum(y[(p == 1)])
    # actual positives (true positive + false negative)
    # c'est simplement la somme des valeurs de y (réalité) à 1.
    ap = np.sum(y)
    return tp / ap

def f1score(p, y):
    pre = precision(p, y)
    rec = recall(p, y)
    return 2 * pre * rec / (pre + rec)

print(precision(p, y))
print(recall(p, y))
print(f1score(p, y))

0.821917808219178
1.0
0.9022556390977443


### A l'aide de l'hyperparameter tuning (random search), trouvez les alpha et lambda qui permettent de maximiser le F1-score. Vous devrez entrainer plusieurs fois votre modele à l'aide de la fonction fit pour trouver ces parametres.

In [19]:
# def hptuning(X, y, theta):
#     a = 0.01
#     l = 0.01
    
#alpha_range = [0.001, 0.002]

def random_search(X, y,
                  occurences=50,
                  alpha_range = [0.001, 0.002],
                  lmbd_range = [0.01, 0.02],
                  iters_range = [100000, 10000000]
                  ):
    a_history = []
    l_history = []
    iter_history = []
    f1_history = []
    J_h_history = []
    for _ in range(occurences):
        tmp_alpha = np.random.uniform(alpha_range[0], alpha_range[1])
        tmp_lmbd = np.random.uniform(lmbd_range[0], lmbd_range[1])
        tmp_iters = np.random.randint(iters_range[0], iters_range[1])
        theta = np.zeros(3, dtype=float).reshape(3, 1)
        theta, J_history = fit(X, y, theta, tmp_alpha, tmp_lmbd, tmp_iters, "L2")
        J_h_history.append(J_history)
        a_history.append(tmp_alpha)
        l_history.append(tmp_lmbd)
        iter_history.append(tmp_iters)
        f1_history.append(f1score(predict(X, theta), y))
    return a_history, l_history, iter_history, f1_history, J_h_history

a_history, l_history, iter_history, f1_history, J_h_history = random_search(X, y)

/sgoinfre/goinfre/Perso/hben-yah/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in double_scalars


In [20]:
print(a_history)

[0.0013944170653433328, 0.0016532356762205484, 0.0010402507160749298, 0.0011092589279398444, 0.0018445686338910779, 0.0016111638951064936, 0.0013435470198427408, 0.001504029711850533, 0.0019139549097483418, 0.0016409286225523712, 0.0018529710176795073, 0.00159895156063965, 0.0018697360570899537, 0.0014020726239134256, 0.0018208836897374036, 0.0014940642698247422, 0.0014041251672340532, 0.0012376111957770235, 0.001886097963897909, 0.0011039784574992499, 0.0019337407624945612, 0.0013590792883841562, 0.0014842083697906496, 0.00115589649643371, 0.0011072330838695086, 0.001859186199653991, 0.0010156027056274026, 0.0018231182070644024, 0.0015990158173501729, 0.001923186077386169, 0.00157036755407911, 0.0010451456026017443, 0.0019641220201322535, 0.001786347452485616, 0.001915084473500455, 0.0016576613453045796, 0.001628381813729107, 0.0010430961028968057, 0.0012701475051024866, 0.001654538336634336, 0.0012129161513785302, 0.001008960975876348, 0.0018866779404926115, 0.0014523552513384388, 0.

## BONUS: Visualisez la frontière de décision (decision boundary) sur le graphe

Pour ceux qui veulent découvrir Matplotlib, il faut ici afficher les données en deux nuages de points distincts (pour les deux classes) sur le même graphe, et aussi trouver une façon de tracer la fonction qui définit la frontière de décision. Amusez-vous bien, et surtout aidez-vous! Voici un exemple de ce que ça devrait donner:  
<img src="figure-2.png">